# Challenge Data Science

[Desafio Alura](https://www.alura.com.br/challenges/data-science-2/semana-03-04-criando-modelo-recomendacao-pyspark?utm_source=ActiveCampaign&utm_medium=email&utm_content=%5BChallenge+Data+Science%5D+T%C3%A1+no+ar%2C++FIRSTNAME&utm_campaign=%5BChallenges%5D+%28Dados+2a+ed+%29+Libera%C3%A7%C3%A3o+da+aula+03e04+%2B+convite+live+code+review&vgo_ee=m974RDr6G5JExwnAIqDaXAA3SuMkJhmkGexv49sZvNU%3D) -  Um projeto de Data Science com PySpark


##  Semana 02

### Imports e carregamento dos Dados

In [126]:
from pyspark.sql import(
    SparkSession, 
    functions as F,
    DataFrame
) 
from pyspark.ml.feature import RFormula
from pyspark.ml.regression import (
    LinearRegression,
     DecisionTreeRegressor,
     RandomForestRegressor
)

import utils

In [127]:
spark = (
    SparkSession
    .builder
    .appName('InsightPlaces-Semana_02')
    .getOrCreate()
)

spark

In [128]:
data_imoveis = (
    spark.read.format('parquet')
    .load('data/semana_02/dataset_tratado/')
)


### Exploração dos Dados

In [129]:
data_imoveis.show(5)

+-----+----------+---------+---------+--------------------+---------+----------+--------------------+-------+------+------------+------------+-----------+----+----------+----+-----+-------+
|andar|area_total|area_util|banheiros|     caracteristicas|   bairro|      zona|          id_anuncio|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|condominio|iptu| tipo|  valor|
+-----+----------+---------+---------+--------------------+---------+----------+--------------------+-------+------+------------+------------+-----------+----+----------+----+-----+-------+
|    3|      43.0|     43.0|      1.0|Academia - Churra...|Paciência|Zona Oeste|d2e3a3aa-09b5-45a...|    2.0|  null|       Usado| Apartamento|Residencial| 1.0|     245.0|null|Venda|15000.0|
|    2|      42.0|     42.0|      1.0|Churrasqueira - P...|Paciência|Zona Oeste|085bab2c-87ad-452...|    2.0|  null|       Usado| Apartamento|Residencial| 1.0|       0.0| 0.0|Venda|15000.0|
|    1|      41.0|     41.0|      1.0|Portaria 24h

In [130]:
print(data_imoveis.count(), len(data_imoveis.columns))

66562 18


In [131]:
data_imoveis.groupBy('zona').count().show()

+------------+-----+
|        zona|count|
+------------+-----+
|  Zona Norte|11897|
|  Zona Oeste|32979|
|Zona Central| 1144|
|    Zona Sul|20531|
|            |   11|
+------------+-----+



In [132]:
''' Deletando colunas tipo_anuncio, tipo_unidade e tipo_uso, pois cada uma
delas possuem valores únicos.

As Colunas area_total e area_util praticamente são iguais e por isso apenas
uma delas será mantida no dataFrame.

'''

data_imoveis = (
    data_imoveis
    .drop('tipo_anuncio')
    .drop('tipo_unidade')
    .drop('tipo_uso')
    .drop('area_total')
)


data_imoveis.printSchema()
data_imoveis.show(5)

root
 |-- andar: long (nullable = true)
 |-- area_util: double (nullable = true)
 |-- banheiros: double (nullable = true)
 |-- caracteristicas: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- id_anuncio: string (nullable = true)
 |-- quartos: double (nullable = true)
 |-- suites: double (nullable = true)
 |-- vaga: double (nullable = true)
 |-- condominio: double (nullable = true)
 |-- iptu: double (nullable = true)
 |-- tipo: string (nullable = true)
 |-- valor: double (nullable = true)

+-----+---------+---------+--------------------+---------+----------+--------------------+-------+------+----+----------+----+-----+-------+
|andar|area_util|banheiros|     caracteristicas|   bairro|      zona|          id_anuncio|quartos|suites|vaga|condominio|iptu| tipo|  valor|
+-----+---------+---------+--------------------+---------+----------+--------------------+-------+------+----+----------+----+-----+-------+
|    3|     43.0|      1.0|

In [133]:
data_imoveis.select('caracteristicas').distinct().show(10, truncate=False)

# 6975

+---------------------------------------------------------------------------------------------------------------------------------------------------------+
|caracteristicas                                                                                                                                          |
+---------------------------------------------------------------------------------------------------------------------------------------------------------+
|Churrasqueira - Condomínio fechado - Portão eletrônico                                                                                                   |
|Playground - Condomínio fechado - Portaria 24h - Animais permitidos                                                                                      |
|Piscina - Academia - Churrasqueira - Playground - Elevador - Condomínio fechado - Portão eletrônico - Portaria 24h - Animais permitidos                  |
|Elevador - Playground - Piscina - Salão de festas - Condomínio 

#### Tratamento da coluna caracteristicas

A colluna caracteristicas tem é uma string com itens separados por ' - ', desta forma será montado um dataframe apenas com está coluna para em seguida agrupar e exibir cada palavra unica

In [134]:
df_caracteristicas = data_imoveis.select('caracteristicas').distinct()

In [135]:
words = df_caracteristicas.select(F.explode( F.split(df_caracteristicas.caracteristicas, ' - ') ).alias('word') )

# 48149

In [136]:
# Palavras unicas da coluna característica

words.groupBy('word').count().show()

+------------------+-----+
|              word|count|
+------------------+-----+
|Condomínio fechado| 5557|
|        Playground| 4839|
| Portão eletrônico| 3859|
|           Piscina| 5016|
|Animais permitidos| 4555|
|      Portaria 24h| 4278|
|          Elevador| 5710|
|          Academia| 4328|
|   Salão de festas| 5407|
|                  |    1|
|     Churrasqueira| 4599|
+------------------+-----+



#### Adicionando features

In [137]:
data_imoveis = (
    data_imoveis
    .withColumn('condominio_fechado', F.when(data_imoveis.caracteristicas.like('%Condomínio fechado%'), 1 ).otherwise(0) )
    .withColumn('playground', F.when(data_imoveis.caracteristicas.like('%Playground%'), 1 ).otherwise(0) )
    .withColumn('portao_eletronico', F.when(data_imoveis.caracteristicas.like('%Portão eletrônico%'), 1 ).otherwise(0) )
    .withColumn('piscina', F.when(data_imoveis.caracteristicas.like('%Piscina%'), 1 ).otherwise(0) )
    .withColumn('permite_animais', F.when(data_imoveis.caracteristicas.like('%Animais permitidos%'), 1 ).otherwise(0) )
    .withColumn('portaria_24h', F.when(data_imoveis.caracteristicas.like('%Portaria 24h%'), 1 ).otherwise(0) )
    .withColumn('elevador', F.when(data_imoveis.caracteristicas.like('%Elevador%'), 1 ).otherwise(0) )
    .withColumn('academia', F.when(data_imoveis.caracteristicas.like('%Academia%'), 1 ).otherwise(0) )
    .withColumn('salao_de_festa', F.when(data_imoveis.caracteristicas.like('%Salão de festas%'), 1 ).otherwise(0) )
    .withColumn('churrasqueira', F.when(data_imoveis.caracteristicas.like('%Churrasqueira%'), 1 ).otherwise(0) )

    .withColumn('tipo_venda', F.when(data_imoveis.tipo.like('%Venda%'), 1 ).otherwise(0) )
    .withColumn('tipo_aluguel', F.when(data_imoveis.tipo.like('%Aluguel%'), 1 ).otherwise(0) )
    .withColumn('tipo_aluguel_e_venda', F.when((data_imoveis.tipo.like('%Aluguel%') & (data_imoveis.tipo.like('%Venda%')) ), 1 ).otherwise(0) )


)

data_imoveis.show(5)
# +-----+---------+---------+--------------------+---------+----------+--------------------+-------+------+----+----------+----+-----+-------+------------------+----------+-----------------+-------+---------------+------------+--------+--------+--------------+-------------+
# |andar|area_util|banheiros|     caracteristicas|   bairro|      zona|          id_anuncio|quartos|suites|vaga|condominio|iptu| tipo|  valor|condominio_fechado|playground|portao_eletronico|piscina|permite_animais|portaria_24h|elevador|academia|salao_de_festa|churrasqueira|
# +-----+---------+---------+--------------------+---------+----------+--------------------+-------+------+----+----------+----+-----+-------+------------------+----------+-----------------+-------+---------------+------------+--------+--------+--------------+-------------+
# |    3|     43.0|      1.0|Academia - Churra...|Paciência|Zona Oeste|d2e3a3aa-09b5-45a...|    2.0|  null| 1.0|     245.0|null|Venda|15000.0|                 1|         1|                1|      0|              1|           1|       0|       1|             1|            1|
# |    2|     42.0|      1.0|Churrasqueira - P...|Paciência|Zona Oeste|085bab2c-87ad-452...|    2.0|  null| 1.0|       0.0| 0.0|Venda|15000.0|                 1|         1|                1|      0|              1|           1|       0|       0|             1|            1|
# |    1|     41.0|      1.0|Portaria 24h - Co...|Guaratiba|Zona Oeste|18d22cbe-1b86-476...|    2.0|  null| 1.0|       0.0| 0.0|Venda|20000.0|                 1|         1|                1|      0|              0|           1|       0|       0|             1|            1|
# |    3|     43.0|      1.0|Churrasqueira - P...|   Cosmos|Zona Oeste|bed8a354-9317-442...|    2.0|  null| 0.0|     285.0|null|Venda|20000.0|                 1|         1|                1|      1|              1|           1|       0|       0|             1|            1|
# |    2|     43.0|      1.0|Academia - Churra...|Guaratiba|Zona Oeste|12a13315-d67f-48f...|    2.0|  null| 1.0|     245.0|null|Venda|15000.0|                 1|         1|                1|      0|              1|           1|       0|       1|             1|            1|
# +-----+---------+---------+--------------------+---------+----------+--------------------+-------+------+----+----------+----+-----+-------+------------------+----------+-----------------+-------+---------------+------------+--------+--------+--------------+-------------+

+-----+---------+---------+--------------------+---------+----------+--------------------+-------+------+----+----------+----+-----+-------+------------------+----------+-----------------+-------+---------------+------------+--------+--------+--------------+-------------+----------+------------+--------------------+
|andar|area_util|banheiros|     caracteristicas|   bairro|      zona|          id_anuncio|quartos|suites|vaga|condominio|iptu| tipo|  valor|condominio_fechado|playground|portao_eletronico|piscina|permite_animais|portaria_24h|elevador|academia|salao_de_festa|churrasqueira|tipo_venda|tipo_aluguel|tipo_aluguel_e_venda|
+-----+---------+---------+--------------------+---------+----------+--------------------+-------+------+----+----------+----+-----+-------+------------------+----------+-----------------+-------+---------------+------------+--------+--------+--------------+-------------+----------+------------+--------------------+
|    3|     43.0|      1.0|Academia - Churra..

In [138]:
data_imoveis.select('playground').groupBy('playground').count().show()

+----------+-----+
|playground|count|
+----------+-----+
|         1|31856|
|         0|34706|
+----------+-----+



#### Tratando dados nulos e NaN's

In [139]:
# Explicitando a coluna label e deletando a coluna id_anuncio

data_imoveis = data_imoveis.withColumnRenamed('valor', 'label').drop('id_anuncio').drop('caracteristicas')

In [140]:
utils.get_count_columns_null_and_empty_values(data_imoveis, True)

-RECORD 0--------------------
 andar                | 0    
 area_util            | 0    
 banheiros            | 0    
 bairro               | 0    
 zona                 | 11   
 quartos              | 0    
 suites               | 5554 
 vaga                 | 3017 
 condominio           | 2944 
 iptu                 | 7741 
 tipo                 | 0    
 label                | 621  
 condominio_fechado   | 0    
 playground           | 0    
 portao_eletronico    | 0    
 piscina              | 0    
 permite_animais      | 0    
 portaria_24h         | 0    
 elevador             | 0    
 academia             | 0    
 salao_de_festa       | 0    
 churrasqueira        | 0    
 tipo_venda           | 0    
 tipo_aluguel         | 0    
 tipo_aluguel_e_venda | 0    



In [141]:
# substituindo os valores nulos
data_imoveis = (
    data_imoveis
    .withColumn('suites', F.when(data_imoveis.suites.isNull(), 0).otherwise(data_imoveis.suites) )
    .withColumn('vaga', F.when(data_imoveis.vaga.isNull(), 0).otherwise(data_imoveis.vaga) )
    .withColumn('condominio', F.when(data_imoveis.condominio.isNull(), 0).otherwise(data_imoveis.condominio) )
    .withColumn('iptu', F.when(data_imoveis.iptu.isNull(), 0).otherwise(data_imoveis.iptu) )
    .withColumn('label', F.when(data_imoveis.label.isNull(), 1294610.18).otherwise(data_imoveis.label) )
    .withColumn('zona', F.when((data_imoveis.zona.isNull()) | (data_imoveis.zona == '') , 'sem zona').otherwise(data_imoveis.zona) )
)

In [142]:
utils.get_count_columns_null_and_empty_values(data_imoveis, True)

-RECORD 0-------------------
 andar                | 0   
 area_util            | 0   
 banheiros            | 0   
 bairro               | 0   
 zona                 | 0   
 quartos              | 0   
 suites               | 0   
 vaga                 | 0   
 condominio           | 0   
 iptu                 | 0   
 tipo                 | 0   
 label                | 0   
 condominio_fechado   | 0   
 playground           | 0   
 portao_eletronico    | 0   
 piscina              | 0   
 permite_animais      | 0   
 portaria_24h         | 0   
 elevador             | 0   
 academia             | 0   
 salao_de_festa       | 0   
 churrasqueira        | 0   
 tipo_venda           | 0   
 tipo_aluguel         | 0   
 tipo_aluguel_e_venda | 0   



#### Preparação dos dados para Machine Learning

In [143]:

data_imoveis = data_imoveis.drop('tipo').drop('bairro')

In [146]:
data_imoveis.write.mode('overwrite').parquet('data/semana_02/dataset_para_ml')

In [144]:
df_final = RFormula(
    formula='label~.',
    featuresCol='features'
).fit(data_imoveis).transform(data_imoveis).select('label', 'features')

In [145]:
df_final.show(truncate=False)

+-------+---------------------------------------------------------------------------------------------------------------+
|label  |features                                                                                                       |
+-------+---------------------------------------------------------------------------------------------------------------+
|15000.0|[3.0,43.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0,1.0,245.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0]       |
|15000.0|(25,[0,1,2,3,7,9,12,13,14,16,17,20,21,22],[2.0,42.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])          |
|20000.0|(25,[0,1,2,3,7,9,12,13,14,17,20,21,22],[1.0,41.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])                 |
|20000.0|(25,[0,1,2,3,7,10,12,13,14,15,16,17,20,21,22],[3.0,43.0,1.0,1.0,2.0,285.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|
|15000.0|[2.0,43.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0,1.0,245.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0]       |
|20000.0|(25,[0,1,2,3,7,

#### Criação e avaliação de modelos